In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df1 = pd.read_csv("../00.data/scraped/meneame_scraped_final_1.csv", encoding="utf-8")
df2 = pd.read_csv("../00.data/scraped/meneame_scraped_final_2.csv", encoding="utf-8")
df3 = pd.read_csv("../00.data/scraped/meneame_scraped_final_3.csv", encoding="utf-8")

In [3]:
df = pd.concat([df1,df2,df3], ignore_index=True)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287191 entries, 0 to 287190
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          287191 non-null  int64 
 1   title            287191 non-null  object
 2   content          287191 non-null  object
 3   full_story_link  287191 non-null  object
 4   meneos           287191 non-null  int64 
 5   clicks           287191 non-null  int64 
 6   karma            287191 non-null  int64 
 7   positive_votes   287191 non-null  int64 
 8   anonymous_votes  287191 non-null  int64 
 9   negative_votes   287191 non-null  int64 
 10  category         287191 non-null  object
 11  comments         287191 non-null  int64 
 12  published_date   287191 non-null  object
 13  user             287191 non-null  object
 14  source           287191 non-null  object
 15  source_link      287191 non-null  object
 16  provincia        287191 non-null  object
 17  comunidad 

In [9]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [11]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [14]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "Gran Canaria": "Las Palmas",
            "Lanzarote": "Las Palmas",
            "Fuerteventura": "Las Palmas",
            "La Graciosa": "Las Palmas",
            "Tenerife": "Santa Cruz de Tenerife",
            "La Gomera": "Santa Cruz de Tenerife",
            "El Hierro": "Santa Cruz de Tenerife",
            "La Palma": "Santa Cruz de Tenerife"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alacant/Alicante", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alacant/Alicante",
            "Alacant": "Alacant/Alicante",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [16]:
# Función para asignar provincias y ccaa del title/content
def asignar_provincia_comunidad(row):
    title = str(row["title"]) if pd.notna(row["title"]) else ""
    content = str(row["content"]) if pd.notna(row["content"]) else ""

    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})

        for provincia in provincias:
            if provincia in title or provincia in content:
                return {"provincia": provincia, "comunidad": comunidad}

        for variante, provincia_estandar in equivalencias.items():
            if variante in title or variante in content:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [18]:
# Aplicar la función fila por fila
df[["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [19]:
df["comunidad"].value_counts()

comunidad
Desconocido             239683
Madrid                   14380
Andalucía                 6524
Comunidad Valenciana      5531
Cataluña                  5432
Castilla y León           3885
Castilla La Mancha        1586
Galicia                   1567
Aragón                    1366
Canarias                  1320
Murcia                    1232
Asturias                  1138
Islas Baleares            1036
Navarra                    540
País Vasco                 532
Extremadura                488
Cantabria                  407
Ceuta                      240
Melilla                    158
La Rioja                   146
Name: count, dtype: int64

In [20]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



# Procesamiento de Categorias


In [22]:
news_category_map = {
    "Geopolítica y Sociedad": ["EEUU", "MADRID", "Rusia", "Japón", "México", "Venezuela", "Cataluña", "Galiza", "Andalucía", "Balears", "Canarias", "CastillaLeón", "deutschland", "Barcelona", "Galicia", "Cantabria", "Aragón", "Málaga", "Roma", "INTERNAC", "Africa", "OrienteMedio", "Cuba"],
    "Movimientos políticos": ["politica", "corruPPción", "corruptcion", "corrupcion", "EleccionesE", "Podemos", "República", "RentaBásica", "Cayetanos", "LuchaDClases", "Anarquismo", "DDHH", "Autonomos", "Justicia7291", "Tribunales", "actualidad", "ZasSmeame", "Campañas", "Fachéame", "VIOLENCIA_G", "prensa", "borbones", "KKKantiPDMS", "AEDE", "procescatalà", "RebeliónMnm", "DchoLaboral", "Bulos", "Desinfórmame", "Madriléame", "Begoñéame"],
    "Conflictos y Guerra": ["Palestina", "Israel", "Siria", "GuerraCivilñ", "ayudaucrania", "Rescates"],
    "Tecnología y Ciencia": ["tecnología", "Informática", "GNU_LINUX", "linux", "startup", "Criptomoneda", "bitcoin", "SectorTIC", "Meteorología", "Física", "genetica", "Neurociencia", "Debian", "retrocomp", "hacking", "Metamaterial", "Micromundo", "atomo", "Impresión3D", "Drones", "netsec", "RaspberryPi", "SysDevs", "ciencia", "astronomia", "ASTRONÁUTICA", "softlibre", "Ubuntu", "android", "smartphone", "Firefox", "Blender", "IOOcia", "Marte", "Visualdata", "INVENTOS", "Numismática", "FÓSILES"],
    "Salud y Medicina": ["salud", "medicina", "Nutrición", "sanidad", "PREVENCIÓN", "HOMEOPATÍA", "VacunaGate", "vacúname", "SANIDAD", "Alimentación", "ebola", "Coronavirus"],
    "Gastronomía": ["Cachopos", "cervecéame"],
    "Negocios y Economía": ["Economía", "banca", "subvenciones", "CrisEnergéti"],
    "Entretenimiento y Cultura": ["ocio", "Cine", "Series", "Videojuegos", "Fotografía", "Música", "cinéfilos", "FolkloreÑ", "Cómics", "Webcomics", "ArteyMercado", "Arte", "Ilustracion", "clásica", "PALABROS", "Covers", "cultura", "Eurovision", "Poesia", "Bushido", "Arquitectura", "rol", "Podcasteros", "Viajar", "CiFi", "Gastronomia", "Clicómics", "Cocíname", "curiosidades", "acapela", "twitter", "reddit", "museo", "Relatocorto", "Autorrelatos", "Reportajes", "Entrevistas", "microrelatos", "Hazlotumismo", "Los12monos", "Pregúntame", "Recomiéndame", "LETREROS", "Escúchame", "Documentales", "STARWARS", "Ochenteando", "UHF", "Preguntas", "Fotomundo", "roleros", "metal", "B.S.O.", "TEMAZOS", "Comiñas", "MundoCelta", "Artículos", "sabíasque", "paléame"],
    "Deportes": ["deportes", "Natación", "Motociclismo", "ciclismo"],
    "Medioambiente y Energía": ["veganismo", "MenteAnimal", "Abandonos", "MUNDO_PROTE", "MAmbiente", "Cambioclima", "Ecología", "Energias", "sequía", "Naturaleza", "Botánica", "PLANTAS", "AgroInfo", "Vivienda", "Animales", "gatos", "Biologia", "Faros", "mascotas", "Cáñamo", "Arañas"],
    "Historia y Humanidades": ["Historia", "Mitología", "Arqueología", "Egiptologia", "Viñetas", "Hallazgos", "Asturias", "Jurídicas", "Etimologia", "Psicología", "Psicópatas", "Filosofía", "Efemérides", "Helénica", "Hemeroteca", "Fragmentos"],
    "Humor y Memes": ["Humor", "memes", "Sarcasmos", "Chorridolias", "Tendenciosos", "Ayuséame", "podríame", "Fakeame", "Fakejóo", "Idiocracia", "meneametoday", "TuMeme", "Cosasguays", "friki", "procrastrina", "NoMundoToday", "Memeneame", "Ardilléame", "Toréame", "OYOYOY", "gilipolleces", "chorradas"],
    "Transporte": ["aviacion", "aviones", "trenes", "DIRIGIBLES", "Submarino", "Motor", "Avionéame", "BARCOS", "Tractores", "movilidad", "alas"],
    "Educación": ["Educación", "idiomas", "Enseñanza", "Literatura", "Libros", "lenguas", "Matemáticas"],
    "Crimen": ["INV", "Gamonal", "Sectas", "Sucesos"],
    "Cuestiones Sociales": ["Asperger", "Crianza", "Drogolegas", "Feminismo", "LGBT", "Derecho", "SEXOLOGÍA", "mujeresenlah", "Abuso_Animal", "religion", "laicismo"]
}


In [23]:
def assign_category_from_text(text):
    assigned_categories = []

    text = text.lower() if isinstance(text, str) else ""

    for category, keywords in news_category_map.items():
        for keyword in keywords:
            if keyword.lower() in text:
                assigned_categories.append(category)
                break 

    return assigned_categories if assigned_categories else ["Miscelaneo"]

df["category"] = df["title"].apply(assign_category_from_text)

In [24]:
category_counts = df["category"].explode().value_counts()

category_counts_df = category_counts.reset_index()
category_counts_df.columns = ["Categoría", "Cantidad"]

import pandas as pd
from IPython.display import display

display(category_counts_df)

,Categoría,Cantidad
0,Miscelaneo,218745
1,Geopolítica y Sociedad,19421
2,Entretenimiento y Cultura,15502
3,Tecnología y Ciencia,6689
4,Crimen,5717
5,Historia y Humanidades,4588
6,Salud y Medicina,3935
7,Movimientos políticos,3805
8,Conflictos y Guerra,3791
9,Medioambiente y Energía,2806


In [35]:
df['category'].value_counts()

category
[Miscelaneo]                                                                218745
[Geopolítica y Sociedad]                                                     16054
[Entretenimiento y Cultura]                                                  12097
[Tecnología y Ciencia]                                                        5034
[Crimen]                                                                      4381
                                                                             ...  
[Movimientos políticos, Medioambiente y Energía, Historia y Humanidades]         1
[Entretenimiento y Cultura, Educación, Cuestiones Sociales]                      1
[Geopolítica y Sociedad, Salud y Medicina, Cuestiones Sociales]                  1
[Geopolítica y Sociedad, Conflictos y Guerra, Deportes]                          1
[Tecnología y Ciencia, Humor y Memes, Crimen]                                    1
Name: count, Length: 265, dtype: int64

In [ ]:
# Guardar en partes para poder subir a Git archivos menores 100MB
df1 = df.iloc[:100_000]
df2 = df.iloc[100_000:200_000]
df3 = df.iloc[200_000:]

df1.to_csv("../00.data/preprocesado/meneame_procesado_1.csv", index=False, encoding="utf-8")
df2.to_csv("../00.data/preprocesado/meneame_procesado_2.csv", index=False, encoding="utf-8")
df3.to_csv("../00.data/preprocesado/meneame_procesado_3.csv", index=False, encoding="utf-8")